In [1]:
# -*- coding: utf-8 -*-
'''
# An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''  # noqa

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

C:\Users\lilanqing\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

In [5]:
# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [9]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [10]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [11]:
print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Instructions for updating:
Use tf.cast instead.
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 350us/step - loss: 1.8330 - acc: 0.3336 - val_loss: 1.7307 - val_acc: 0.3540
Q 64+96   T 160  ☒ 111 
Q 649+444 T 1093 ☒ 100 
Q 821+76  T 897  ☒ 150 
Q 43+767  T 810  ☒ 647 
Q 444+938 T 1382 ☒ 115 
Q 77+237  T 314  ☒ 143 
Q 233+65  T 298  ☒ 743 
Q 4+771   T 775  ☒ 111 
Q 393+132 T 525  ☒ 133 
Q 707+162 T 869  ☒ 105 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 285us/step - loss: 1.6473 - acc: 0.3853 - val_loss: 1.5667 - val_acc: 0.4118
Q 13+125  T 138  ☒ 23  
Q 49+812  T 861  ☒ 899 
Q 905+5   T 910  ☒ 966 
Q 490+896 T 1386 ☒ 1464
Q 167+41  T 208  ☒ 778 
Q 533+43  T 576  ☒ 467 
Q 66+313  T 379  ☒ 467 
Q 41+188  T 229  ☒ 222 
Q 61+838  T 899  ☑

45000/45000 [==============================] - 12s 268us/step - loss: 0.1646 - acc: 0.9632 - val_loss: 0.1618 - val_acc: 0.9618
Q 87+711  T 798  ☒ 898 
Q 807+683 T 1490 ☑ 1490
Q 63+396  T 459  ☒ 469 
Q 11+559  T 570  ☑ 570 
Q 34+34   T 68   ☑ 68  
Q 29+54   T 83   ☑ 83  
Q 770+47  T 817  ☑ 817 
Q 11+288  T 299  ☒ 399 
Q 81+736  T 817  ☑ 817 
Q 979+149 T 1128 ☑ 1128

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 266us/step - loss: 0.1528 - acc: 0.9624 - val_loss: 0.1341 - val_acc: 0.9708
Q 586+91  T 677  ☑ 677 
Q 272+51  T 323  ☑ 323 
Q 518+39  T 557  ☑ 557 
Q 962+96  T 1058 ☑ 1058
Q 59+187  T 246  ☑ 246 
Q 93+387  T 480  ☑ 480 
Q 987+311 T 1298 ☑ 1298
Q 776+93  T 869  ☑ 869 
Q 691+25  T 716  ☑ 716 
Q 603+15  T 618  ☑ 618 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 12s 256us/step - loss: 0.0336 - acc: 0.9931 - val_loss: 0.0396 - val_acc: 0.9902
Q 34+34   T 68   ☑ 68  
Q 705+51  T 756  ☑ 756 
Q 430+615 T 1045 ☑ 1045
Q 18+88   T 106  ☑ 106 
Q 664+79  T 743  ☑ 743 
Q 459+62  T 521  ☑ 521 
Q 255+700 T 955  ☑ 955 
Q 65+827  T 892  ☑ 892 
Q 78+938  T 1016 ☑ 1016
Q 19+765  T 784  ☑ 784 

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 258us/step - loss: 0.0657 - acc: 0.9797 - val_loss: 0.0367 - val_acc: 0.9918
Q 406+701 T 1107 ☑ 1107
Q 27+59   T 86   ☑ 86  
Q 77+787  T 864  ☑ 864 
Q 45+920  T 965  ☑ 965 
Q 19+60   T 79   ☑ 79  
Q 22+808  T 830  ☑ 830 
Q 592+772 T 1364 ☑ 1364
Q 69+60   T 129  ☑ 129 
Q 981+45  T 1026 ☑ 1026
Q 344+45  T 389  ☑ 389 

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 12s 258us/step - loss: 0.0104 - acc: 0.9986 - val_loss: 0.0186 - val_acc: 0.9953
Q 80+369  T 449  ☑ 449 
Q 200+83  T 283  ☑ 283 
Q 457+11  T 468  ☑ 468 
Q 282+33  T 315  ☑ 315 
Q 318+555 T 873  ☑ 873 
Q 983+82  T 1065 ☑ 1065
Q 518+398 T 916  ☑ 916 
Q 216+338 T 554  ☑ 554 
Q 923+571 T 1494 ☑ 1494
Q 787+22  T 809  ☑ 809 

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 256us/step - loss: 0.0141 - acc: 0.9971 - val_loss: 0.0518 - val_acc: 0.9830
Q 396+559 T 955  ☑ 955 
Q 866+883 T 1749 ☑ 1749
Q 94+170  T 264  ☑ 264 
Q 13+432  T 445  ☑ 445 
Q 618+62  T 680  ☑ 680 
Q 165+73  T 238  ☑ 238 
Q 889+835 T 1724 ☑ 1724
Q 856+79  T 935  ☑ 935 
Q 717+76  T 793  ☑ 793 
Q 857+20  T 877  ☑ 877 

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 12s 270us/step - loss: 0.0322 - acc: 0.9899 - val_loss: 0.0570 - val_acc: 0.9789
Q 90+355  T 445  ☑ 445 
Q 2+855   T 857  ☑ 857 
Q 95+431  T 526  ☑ 526 
Q 79+795  T 874  ☑ 874 
Q 919+108 T 1027 ☑ 1027
Q 19+67   T 86   ☑ 86  
Q 758+322 T 1080 ☑ 1080
Q 545+9   T 554  ☑ 554 
Q 997+66  T 1063 ☑ 1063
Q 184+274 T 458  ☑ 458 

--------------------------------------------------
Iteration 58
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 267us/step - loss: 0.0157 - acc: 0.9960 - val_loss: 0.0289 - val_acc: 0.9919
Q 404+740 T 1144 ☑ 1144
Q 26+89   T 115  ☑ 115 
Q 408+902 T 1310 ☑ 1310
Q 481+71  T 552  ☑ 552 
Q 500+374 T 874  ☑ 874 
Q 589+52  T 641  ☑ 641 
Q 908+42  T 950  ☑ 950 
Q 95+132  T 227  ☑ 227 
Q 34+864  T 898  ☑ 898 
Q 841+30  T 871  ☑ 871 

--------------------------------------------------
Iteration 59
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

...

45000/45000 [==============================] - 12s 258us/step - loss: 0.0019 - acc: 0.9998 - val_loss: 0.0039 - val_acc: 0.9992
Q 605+85  T 690  ☑ 690 
Q 117+744 T 861  ☑ 861 
Q 824+108 T 932  ☑ 932 
Q 852+46  T 898  ☑ 898 
Q 55+92   T 147  ☑ 147 
Q 71+714  T 785  ☑ 785 
Q 16+514  T 530  ☑ 530 
Q 1+221   T 222  ☑ 222 
Q 80+369  T 449  ☑ 449 
Q 910+873 T 1783 ☑ 1783

--------------------------------------------------
Iteration 142
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 255us/step - loss: 9.2914e-04 - acc: 1.0000 - val_loss: 0.0031 - val_acc: 0.9993
Q 544+83  T 627  ☑ 627 
Q 974+51  T 1025 ☑ 1025
Q 935+59  T 994  ☑ 994 
Q 9+204   T 213  ☑ 213 
Q 646+53  T 699  ☑ 699 
Q 907+64  T 971  ☑ 971 
Q 767+32  T 799  ☑ 799 
Q 79+795  T 874  ☑ 874 
Q 336+62  T 398  ☑ 398 
Q 928+55  T 983  ☑ 983 

--------------------------------------------------
Iteration 143
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/4500

45000/45000 [==============================] - 12s 257us/step - loss: 6.1624e-04 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 0.9992
Q 31+155  T 186  ☑ 186 
Q 123+253 T 376  ☑ 376 
Q 509+202 T 711  ☑ 711 
Q 568+79  T 647  ☑ 647 
Q 646+64  T 710  ☑ 710 
Q 291+43  T 334  ☑ 334 
Q 978+676 T 1654 ☑ 1654
Q 17+711  T 728  ☑ 728 
Q 12+214  T 226  ☑ 226 
Q 860+293 T 1153 ☑ 1153

--------------------------------------------------
Iteration 156
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 259us/step - loss: 0.0191 - acc: 0.9945 - val_loss: 0.1454 - val_acc: 0.9555
Q 181+7   T 188  ☑ 188 
Q 79+32   T 111  ☑ 111 
Q 486+57  T 543  ☒ 552 
Q 300+99  T 399  ☑ 399 
Q 67+289  T 356  ☑ 356 
Q 126+498 T 624  ☑ 624 
Q 460+926 T 1386 ☑ 1386
Q 476+867 T 1343 ☑ 1343
Q 88+119  T 207  ☒ 217 
Q 946+114 T 1060 ☑ 1060

--------------------------------------------------
Iteration 157
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/4500

45000/45000 [==============================] - 12s 257us/step - loss: 0.0345 - acc: 0.9899 - val_loss: 0.0208 - val_acc: 0.9931
Q 632+58  T 690  ☑ 690 
Q 93+808  T 901  ☑ 901 
Q 471+139 T 610  ☑ 610 
Q 1+419   T 420  ☑ 420 
Q 765+273 T 1038 ☑ 1038
Q 23+78   T 101  ☑ 101 
Q 482+106 T 588  ☑ 588 
Q 9+624   T 633  ☑ 633 
Q 650+405 T 1055 ☑ 1055
Q 772+352 T 1124 ☑ 1124

--------------------------------------------------
Iteration 170
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 256us/step - loss: 0.0035 - acc: 0.9993 - val_loss: 0.0047 - val_acc: 0.9988
Q 231+37  T 268  ☑ 268 
Q 580+401 T 981  ☑ 981 
Q 672+506 T 1178 ☑ 1178
Q 32+62   T 94   ☑ 94  
Q 375+280 T 655  ☑ 655 
Q 93+71   T 164  ☑ 164 
Q 21+670  T 691  ☑ 691 
Q 96+655  T 751  ☑ 751 
Q 447+86  T 533  ☑ 533 
Q 685+39  T 724  ☑ 724 

--------------------------------------------------
Iteration 171
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=

45000/45000 [==============================] - 12s 256us/step - loss: 0.0013 - acc: 0.9999 - val_loss: 0.0039 - val_acc: 0.9988
Q 1+651   T 652  ☑ 652 
Q 8+782   T 790  ☑ 790 
Q 958+681 T 1639 ☑ 1639
Q 526+316 T 842  ☑ 842 
Q 595+3   T 598  ☑ 598 
Q 79+498  T 577  ☑ 577 
Q 946+548 T 1494 ☑ 1494
Q 4+256   T 260  ☑ 260 
Q 77+852  T 929  ☑ 929 
Q 44+318  T 362  ☑ 362 

--------------------------------------------------
Iteration 184
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 258us/step - loss: 6.8359e-04 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 0.9991
Q 85+95   T 180  ☑ 180 
Q 84+570  T 654  ☑ 654 
Q 528+91  T 619  ☑ 619 
Q 374+688 T 1062 ☑ 1062
Q 67+272  T 339  ☑ 339 
Q 42+68   T 110  ☑ 110 
Q 497+90  T 587  ☑ 587 
Q 920+655 T 1575 ☑ 1575
Q 60+425  T 485  ☑ 485 
Q 55+830  T 885  ☑ 885 

--------------------------------------------------
Iteration 185
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/4500

45000/45000 [==============================] - 12s 256us/step - loss: 3.9422e-04 - acc: 1.0000 - val_loss: 0.0020 - val_acc: 0.9994
Q 825+237 T 1062 ☑ 1062
Q 789+253 T 1042 ☑ 1042
Q 589+7   T 596  ☑ 596 
Q 18+522  T 540  ☑ 540 
Q 688+549 T 1237 ☑ 1237
Q 94+57   T 151  ☑ 151 
Q 4+729   T 733  ☑ 733 
Q 635+2   T 637  ☑ 637 
Q 960+89  T 1049 ☑ 1049
Q 70+858  T 928  ☑ 928 

--------------------------------------------------
Iteration 198
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 256us/step - loss: 3.5908e-04 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 0.9994
Q 561+935 T 1496 ☑ 1496
Q 313+860 T 1173 ☑ 1173
Q 78+732  T 810  ☑ 810 
Q 35+913  T 948  ☑ 948 
Q 956+42  T 998  ☑ 998 
Q 621+358 T 979  ☑ 979 
Q 852+359 T 1211 ☑ 1211
Q 254+26  T 280  ☑ 280 
Q 59+897  T 956  ☑ 956 
Q 794+805 T 1599 ☑ 1599

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

In [14]:
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'addition_rnn_trained_model.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at D:\lilanqing\CODE\Python_learn\keras-study\saved_models\addition_rnn_trained_model.h5 
